In [1]:
from utils.general import info, ok, warning, id2file

### Label Data

2022-03-16 15:06:57.322995 [ INFO  ] len(relevant_set) = 39
2022-03-16 15:06:57.323522 [ INFO  ] len(irrelevant)   = 45


In [9]:
import os
import re

second_round_data = [linea.strip().split(';') for linea in  open('second_roud_labels.csv','r').read().splitlines()]
irrelevant_set = set([id_ for id_,label in second_round_data if label=='I'])
relevant_set = set([id_ for id_,label in second_round_data if label=='R'])


# Loading new_data
new_data = [line.split(';') for line in open('new_data.csv').read().splitlines()]
relevant_set = relevant_set.union(set([id_ for id_,label in new_data if label.strip()=='R']))
irrelevant_set = irrelevant_set.union(set([id_ for id_,label in new_data if label.strip()=='I']))

# Loading original data
DP_examples_dirpath = '/home/ec2-user/SageMaker/mariano/notebooks/04. Model of DP/DP-relevant articles/'

first_data = []
for dirpath, dirnames, filenames in os.walk(DP_examples_dirpath):
    for filename in filenames:
        content = open(os.path.join(dirpath,filename),'r').read()
        ids = re.findall('/docview/([^/]*)/',content)
        relevant_set = relevant_set.union(set(ids))
    
# articles containg DP and Canada from that period, that were not deteted by Serperi
GM_dp_dirpath = '/home/ec2-user/SageMaker/data/GM_DP_and_Canada1945_1967/'

files = os.listdir(GM_dp_dirpath)

irrelevant_set = irrelevant_set.union([file_[:-4] for file_ in files if file_[:-4] not in relevant_set and file_.endswith('.xml')])

not_found=[]
for id_ in list(relevant_set)+list(irrelevant_set):
    if id2file(id_) is None:
        not_found.append(id_)
print(f'Not found: {not_found}')
for id_ in not_found:
    relevant_set = relevant_set.difference(set(not_found))
    irrelevant_set = irrelevant_set.difference(set(not_found))
    
info(f'len(relevant_set)   = {len(relevant_set)}')
info(f'len(irrelevant_set) = {len(irrelevant_set)}')


Not found: ['2122281371', '1238440920', '1242257052', '1151348424', '1411697642', '1136691129', '1238204962', '1143160388', '2459666609', '1239753620', '2459964104', '2122279956', '1222379804']
2022-03-16 15:07:40.837482 [ INFO  ] len(relevant_set)   = 581
2022-03-16 15:07:40.837607 [ INFO  ] len(irrelevant_set) = 6523


In [12]:
from utils.tdmstudio import TDMStudio
corpus = [TDMStudio.get_title_and_text(id2file(id_)) for id_ in list(relevant_set)+list(irrelevant_set)]
corpus = [f'{title}. {text}' for title,text in corpus]

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from utils.models import tokenize
from utils.tdmstudio import TDMStudio
import spacy
import string

# AUXs
nlp = spacy.load('en_core_web_sm', disable=['textcat', 'parser','ner'])

def remove_punctuation(word):
    return ''.join([char for char in word if not char in string.punctuation+' '])

def tokenize(str_):
    tokens = [word.lemma_.lower() for word in nlp(str_) if not word.is_stop]
    tokens = [word.replace('\n', '') for word in tokens if not word.isnumeric() and len(remove_punctuation(word))!=0]
    return tokens

# STOPWORDS
stopwords = nlp.Defaults.stop_words
invalid = set([sw for sw in stopwords if any([token for token in tokenize(sw) if not token in stopwords ])]) # ['‘ve', "'m", '’ve', "'ve", '’m', '‘m', '‘d', '‘ll']
stopwords = set(stopwords.difference(invalid))

#MODEL
vectorizer = TfidfVectorizer(
                             input='content',
                             lowercase=True,
                             preprocessor=None,
                             tokenizer=tokenize,
                             analyzer='word',
                             stop_words=list(stopwords),
#                              token_pattern=r"(?u)\b\w\w+\b", #selects tokens of 2 or more alphanumeric char (punctuation is completely ignored and treated as token separator)
#                                                              # UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None
    
                             ngram_range=(1,1), #lower and upper boundary of the range of n-values for different n-grams.
                             max_df=1.0, #ignore terms that have a document frequency strictly higher than given threshold
                             min_df=0.001, #ignore terms that have a document frequency strictly lower than given threshold
                             max_features=50000, #build a vocabulary that only considers the top max_features ordered by term frequency acoss the corpus
                             vocabulary=None, #vocabulary is determined from the input documents
                             norm='l2',
                             use_idf=False,
                             )

X = vectorizer.fit_transform(corpus)

In [14]:
import numpy as np
y = np.zeros(shape=(X.shape[0],))
y[:len(relevant_set)]=1
y

array([1., 1., 1., ..., 0., 0., 0.])

In [ ]:
import pandas as pd

from sklearn.model_selection import cross_validate
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
parameters = {'kernel':['linear','rbf','poly'], 'C':[0.1, 0.5, 1, 5, 7, 12, 15, 30, 45]}
svc = SVC()

# parameters = {'C':[1,2,3,4,5,6], 'degree':[1,2,3,4,5,6,7]}
# svc = SVC(kernel='poly')
clf = GridSearchCV(svc, parameters,scoring='f1', cv=5,verbose=4)

results = clf.fit(X,y)
pd.DataFrame(results.cv_results_)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV 1/5] END ..............C=0.1, kernel=linear;, score=0.813 total time=  31.9s
[CV 2/5] END ..............C=0.1, kernel=linear;, score=0.756 total time=  31.2s
[CV 3/5] END ..............C=0.1, kernel=linear;, score=0.778 total time=  31.2s
[CV 4/5] END ..............C=0.1, kernel=linear;, score=0.818 total time=  31.3s
[CV 5/5] END ..............C=0.1, kernel=linear;, score=0.742 total time=  30.9s
[CV 1/5] END .................C=0.1, kernel=rbf;, score=0.537 total time=  36.6s
[CV 2/5] END .................C=0.1, kernel=rbf;, score=0.563 total time=  36.3s
[CV 3/5] END .................C=0.1, kernel=rbf;, score=0.510 total time=  36.5s
[CV 4/5] END .................C=0.1, kernel=rbf;, score=0.549 total time=  37.2s
[CV 5/5] END .................C=0.1, kernel=rbf;, score=0.490 total time=  36.3s
[CV 1/5] END ................C=0.1, kernel=poly;, score=0.000 total time= 1.1min
[CV 2/5] END ................C=0.1, kernel=poly

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,25.904436,0.359086,5.398545,0.104160,0.1,linear,"{'C': 0.1, 'kernel': 'linear'}",0.813397,0.755981,0.777778,0.818182,0.742268,0.781521,0.030223,17
1,30.856181,0.350201,5.721601,0.089998,0.1,rbf,"{'C': 0.1, 'kernel': 'rbf'}",0.537500,0.562874,0.509554,0.548780,0.490323,0.529806,0.026389,26
2,53.019771,0.387060,11.954176,0.297902,0.1,poly,"{'C': 0.1, 'kernel': 'poly'}",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,27
3,19.553209,0.291093,4.026551,0.133296,0.5,linear,"{'C': 0.5, 'kernel': 'linear'}",0.854545,0.817391,0.850467,0.842553,0.838095,0.840610,0.012966,9
4,27.235549,0.432903,4.998342,0.151213,0.5,rbf,"{'C': 0.5, 'kernel': 'rbf'}",0.833333,0.778281,0.821256,0.842105,0.790000,0.812995,0.024751,16
5,53.666411,0.438137,12.159250,0.280636,0.5,poly,"{'C': 0.5, 'kernel': 'poly'}",0.662983,0.615385,0.632184,0.691099,0.619883,0.644307,0.028700,25
6,18.436344,0.409448,3.818246,0.146319,1,linear,"{'C': 1, 'kernel': 'linear'}",0.864629,0.829694,0.862385,0.843882,0.844037,0.848925,0.013015,1
7,27.187717,0.544096,5.042303,0.159893,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.855856,0.817391,0.855814,0.854701,0.843602,0.845473,0.014778,8
8,54.371512,0.694372,12.171955,0.311285,1,poly,"{'C': 1, 'kernel': 'poly'}",0.753769,0.719212,0.758974,0.809302,0.744681,0.757188,0.029426,24
9,18.464932,0.327564,3.853887,0.112577,5,linear,"{'C': 5, 'kernel': 'linear'}",0.829694,0.847458,0.818182,0.823045,0.805556,0.824787,0.013817,15


### Best model's performance using CV

In [16]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(SVC(C=1,kernel='linear'),X,y,cv=5,scoring=['f1','precision','recall','accuracy'])
for metric in cv_results:
    print(f'{metric:10}: {np.average(cv_results[metric]):5.4f}')

fit_time  : 18.1400
score_time: 3.6862
test_f1   : 0.8489
test_precision: 0.8749
test_recall: 0.8261
test_accuracy: 0.9759


### Training Best model on all the labeled data for prediction

In [20]:
model = SVC(C=1, kernel='linear' ) #FALTÓ: ,probability=True
model.fit(X,y)

SVC(C=1, kernel='linear')

### Loading all unlabeled data (excluding labeled data)

In [17]:
GM_all_part1 = '/home/ec2-user/SageMaker/data/GM_all_1945_1956/'
GM_all_part2 = '/home/ec2-user/SageMaker/data/GM_all_1957-1967/'

files = [GM_all_part1+filename for filename in os.listdir(GM_all_part1) if not filename[:-4] in irrelevant_set.union(relevant_set)]
files += [GM_all_part2+filename for filename in os.listdir(GM_all_part2) if not filename[:-4] in irrelevant_set.union(relevant_set)]

info(f'Predictions needed over {len(files)} files.')


2022-03-16 17:51:39.799327 [ INFO  ] Predictions needed over 2050764 files.


In [22]:
!echo $[ $(ls /home/ec2-user/SageMaker/data/GM_all_1945_1956/ | grep xml | wc -l ) + $(ls /home/ec2-user/SageMaker/data/GM_all_1957-1967/ | grep xml | wc -l ) ]

2057868


### Storing best model and vectorizer (TfIdf vectorizer)

In [25]:
from joblib import dump, load
dump(model, 'cache/model.joblib')
dump(vectorizer, 'cache/vectorizer.joblib') 

# model = load('cache/model.joblib')
# vectorizer = load('cache/vectorizer.joblib')

['cache/vectorizer.joblib']

### Freeing space in memory

In [24]:
total = len(corpus)
for idx in range(total):
    del(corpus[total-1-idx])
del(clf)
del(y)
del(X)

In [28]:
# # del(X)
# # del(y)
# aux = files[-100:]
# corpus = [TDMStudio.get_title_and_text(file_) for file_ in aux]
# corpus = [f'{title}. {text}' for title,text in corpus]

# X = vectorizer.transform(corpus)
# yhat = model.predict(X)
# yhat.shape

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [32]:
# files_copy = files.copy()

In [67]:
files = files_copy.copy()
len(files)

2050764

In [68]:
info(len(files))
info(len(files_copy))

2022-03-16 21:37:16.019226 [ INFO  ] 2050764
2022-03-16 21:37:16.019416 [ INFO  ] 2050764


### Storing empty predictions files from all suitable news articles

In [ ]:
## import numpy as np
import pickle

predictions_dirpath = './predictions/'
progress_file = './predictions/v3_progress_file.txt'
processed=0
info('Starting...')
while len(files)>0:
    cap=10000 if len(files)>=10000 else len(files)
    
    aux = files[-cap:]
    
    corpus = []
    for file_ in aux:
        id_ = file_.split('/')[-1][:-4]
        title,text = TDMStudio.get_title_and_text(file_)
        if not title is None and not text is None:            
            corpus.append((id_,f'{title}. {text}'))
            
    X = vectorizer.transform([text for _,text in corpus])
    yhat = model.predict(X)
    writer = (open(progress_file,'a'))
    for item,label in zip(corpus,yhat):
        id_,text = item
        writer.write(id_+'\n')
        file_= predictions_dirpath +id_ + '_v3.p' 
        if label==1:
            pickle.dump(np.zeros(shape=(3,)),open(file_,'wb'))
    writer.close()
    processed+=cap
    while (cap)!=0:
        del(files[-1])
        cap-=1
        
    del(X)
    del(yhat)
    del(aux)
    del(corpus)
    del(writer)
    if processed%20000==0:
        info(f'Processed {processed:07,}')
info('Done!')
processed

2022-03-16 21:37:18.261704 [ INFO  ] Starting...
2022-03-16 21:45:52.737059 [ INFO  ] Processed 020,000
2022-03-16 21:55:04.217567 [ INFO  ] Processed 040,000
2022-03-16 22:04:07.564664 [ INFO  ] Processed 060,000
2022-03-16 22:13:01.420476 [ INFO  ] Processed 080,000
2022-03-16 22:21:48.540774 [ INFO  ] Processed 100,000
2022-03-16 22:30:42.605117 [ INFO  ] Processed 120,000


In [61]:
processed

217000

In [70]:
processed

2050764

In [62]:
print('hello world')

hello world


In [46]:
# len(files_copy)

2050764

In [55]:
# title,text = TDMStudio.get_title_and_text(id2file('1270315270'))
# _ = [f'{title}. {text}']
# model.predict(vectorizer.transform(_))

array([1.])

In [57]:
# TDMStudio.get_title_and_text(id2file('1270315270'))
                             

('Bid for Eastern Europeans',
 "\n\n\n\xa0\n\n\n\nBid for Eastern Europeans\n\n\nImmigrants to tell politics\n\n\nFrom the Ottawa Bureau of The Globe and Mail\n\n\nOTTAWA--Immigrants are being asked to list the political and social groups they have belonged to in an administrative change aimed at opening the gates little wider to immigration from Eastern Europe\n\n\nThe information is sought on new application forms that came into use month ago Until now the immigration de partment depended solely on the authorities in the immigrant s country to help it screen political security risks One result department sources said yesterday was to restrict im- 4\n\n\nmigration from countries that do not provide this kind of information to the sponsored dependents of peo ple already in Canada With the new form the depart ment feels it is able to handle ap plications from other relatives of Canadians from these countries The move will not open the floodgates to Immigration from Eastern Europe since 